In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import emcee
from astropy.time import Time
from astropy import constants as const
from matplotlib.collections import LineCollection
from matplotlib import cm
import astropy.units as u

# Load data
file_path = "AssigmentDistance2SgrA_mockObservations.csv"
df = pd.read_csv(file_path)
time = Time(pd.to_datetime(df['Time of Observation'])).jyear
delta_ra = df['Delta R.A. [as] (0.01as error)'].values
delta_dec = df['Delta Dec. [as] (0.01as error)'].values
vz = df['vz [km/s] (10km/s error)'].values

# Constants
as_to_rad = np.pi / (180.0 * 3600.0)
G = const.G.cgs.value
Msun = const.M_sun.cgs.value
AU = const.au.cgs.value
kpc_to_cm = 1e3 * const.pc.cgs.value

# Full Keplerian orbit model to predict observables
def kepler_model_full(params, t):
    """
    Predicts observables (delta_RA, delta_Dec, vz) for a full Keplerian orbit.

    Parameters
    ----------
    params : list
        [M (Msun), D (kpc), a (AU), e, i (deg), Omega (deg), omega (deg), T0 (jyear)]
    t : array
        Observation times in Julian years

    Returns
    -------
    x_model : np.ndarray
        Predicted delta R.A. values in arcseconds
    y_model : np.ndarray
        Predicted delta Dec. values in arcseconds
    vz_model : np.ndarray
        Predicted line-of-sight velocity in km/s
    """
    M, D, a_AU, e, i_deg, Omega_deg, omega_deg, T0_jyear = params

    # Prior checks for numerical stability before calculations
    if not (1e6 < M < 5e6 and 6 < D < 10 and 0.1 < a_AU < 200 and 0 <= e < 1.0 and
            0 <= i_deg <= 180 and 0 <= Omega_deg <= 360 and 0 <= omega_deg <= 360 and
            2000 < T0_jyear < 2030): # T0 range to cover periapsis within observation period roughly
        return (np.full_like(t, np.nan), np.full_like(t, np.nan), np.full_like(t, np.nan))

    # Convert to working units (cgs)
    M_cgs = M * Msun
    D_cm = D * kpc_to_cm
    a_cm = a_AU * AU
    i_rad = np.radians(i_deg)
    Omega_rad = np.radians(Omega_deg)
    omega_rad = np.radians(omega_deg)

    # Gravitational parameter in cgs
    mu = G * M_cgs

    # Mean motion (rad / s)
    n_mean = np.sqrt(mu / a_cm**3)

    # Initialize model arrays
    x_model = np.zeros_like(t)
    y_model = np.zeros_like(t)
    vz_model = np.zeros_like(t)

    # Iterate over each observation time
    for j, current_t_jyear in enumerate(t):
        # Time from periapsis passage in seconds
        delta_t_s = (current_t_jyear - T0_jyear) * (365.25 * 24 * 3600)

        # Mean anomaly (M)
        M_anom = n_mean * delta_t_s
        M_anom = np.fmod(M_anom, 2 * np.pi) # Wrap to [0, 2pi)

        # Solve Kepler's equation for Eccentric Anomaly (E)
        # E - e * sin(E) = M
        E_guess = M_anom
        for _ in range(100): # Iterate a few times for convergence
            f = E_guess - e * np.sin(E_guess) - M_anom
            f_prime = 1 - e * np.cos(E_guess)
            if f_prime == 0: # Avoid division by zero for highly eccentric orbits near periapsis
                break
            E_new = E_guess - f / f_prime
            if np.abs(E_new - E_guess) < 1e-9: # Convergence criterion
                E_guess = E_new
                break
            E_guess = E_new
        E = E_guess

        # Position in orbital plane (perifocal frame)
        x_perifocal = a_cm * (np.cos(E) - e)
        y_perifocal = a_cm * np.sqrt(1 - e**2) * np.sin(E)

        # Velocity in orbital plane (perifocal frame)
        Edot = n_mean / (1 - e * np.cos(E))
        vx_perifocal = -a_cm * np.sin(E) * Edot
        vy_perifocal = a_cm * np.sqrt(1 - e**2) * np.cos(E) * Edot

        # Rotate to inertial frame (equatorial J2000 for RA/Dec projection)
        # Standard rotations: R_z(Omega) @ R_x(i) @ R_z(omega)
        cos_omega, sin_omega = np.cos(omega_rad), np.sin(omega_rad)
        cos_i, sin_i = np.cos(i_rad), np.sin(i_rad)
        cos_Omega, sin_Omega = np.cos(Omega_rad), np.sin(Omega_rad)

        # Rotation matrix components
        P11 = cos_Omega * cos_omega - sin_Omega * sin_omega * cos_i
        P12 = -cos_Omega * sin_omega - sin_Omega * cos_omega * cos_i
        P21 = sin_Omega * cos_omega + cos_Omega * sin_omega * cos_i
        P22 = -sin_Omega * sin_omega + cos_Omega * cos_omega * cos_i
        P31 = sin_omega * sin_i
        P32 = cos_omega * sin_i

        # Position in inertial frame (cm)
        x_inertial = P11 * x_perifocal + P12 * y_perifocal
        y_inertial = P21 * x_perifocal + P22 * y_perifocal
        z_inertial = P31 * x_perifocal + P32 * y_perifocal # Z is along line-of-sight

        # Velocity in inertial frame (cm/s)
        vx_inertial = P11 * vx_perifocal + P12 * vy_perifocal
        vy_inertial = P21 * vx_perifocal + P22 * vy_perifocal
        vz_inertial = P31 * vx_perifocal + P32 * vy_perifocal # Z velocity is radial velocity

        # Convert to observables: RA (arcsec), Dec (arcsec), vz (km/s)
        # Note: Delta RA is often defined as positive west, which means negative along celestial x-axis
        x_model[j] = -x_inertial / D_cm * (180 * 3600 / np.pi)
        y_model[j] = y_inertial / D_cm * (180 * 3600 / np.pi)
        vz_model[j] = vz_inertial / 1e5 # cm/s to km/s

    return x_model, y_model, vz_model

# Log-likelihood function
def log_likelihood(params, t, x, y, vz):
    x_model, y_model, vz_model = kepler_model_full(params, t)
    if np.any(np.isnan(x_model)) or np.any(np.isnan(y_model)) or np.any(np.isnan(vz_model)):
        return -np.inf
    err_pos = 0.01  # positional error in arcsec
    err_vz = 10     # velocity error in km/s
    chi2 = np.sum(((x - x_model) / err_pos) ** 2 + ((y - y_model) / err_pos) ** 2 + ((vz - vz_model) / err_vz) ** 2)
    return -0.5 * chi2

# Log-prior
def log_prior_full(params):
    M, D, a_AU, e, i_deg, Omega_deg, omega_deg, T0_jyear = params

    # Priors based on typical values for S2-like stars
    if not (1e6 < M < 5e6): return -np.inf # Mass of Sgr A*
    if not (6 < D < 10): return -np.inf # Distance to Sgr A*
    if not (5 < a_AU < 25): return -np.inf # Semi-major axis (S2 is ~15 AU)
    if not (0.5 <= e < 0.99): return -np.inf # Eccentricity (S2 is ~0.88)
    if not (90 <= i_deg <= 180): return -np.inf # Inclination (S2 is ~133 deg)
    if not (0 <= Omega_deg <= 360): return -np.inf # Longitude of Node
    if not (0 <= omega_deg <= 360): return -np.inf # Argument of Periapsis
    if not (2017 < T0_jyear < 2019): return -np.inf # Periapsis epoch (S2 is ~2018.38)

    return 0.0 # Uniform prior within bounds

# Log-probability
def log_probability(params, t, x, y, vz):
    lp = log_prior_full(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, t, x, y, vz)

# Initial guess for the 8 parameters (M, D, a, e, i, Omega, omega, T0)
# Using values close to known S2 orbit
initial = np.array([4.0e6, 8.0, 15.0, 0.88, 133.0, 228.0, 66.0, 2018.38])

ndim = len(initial)
nwalkers = 2 * ndim # Good practice to have at least 2*ndim walkers

# Perturb initial positions for walkers
pos = initial + 1e-4 * initial * np.random.randn(nwalkers, ndim)
for i in range(nwalkers):
    while not np.isfinite(log_probability(pos[i], time, delta_ra, delta_dec, vz)):
        pos[i] = initial + 1e-4 * initial * np.random.randn(ndim)


print(f"Running MCMC with {nwalkers} walkers and {ndim} dimensions...")
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(time, delta_ra, delta_dec, vz))
# It's recommended to run a short burn-in first, then the main run
state = sampler.run_mcmc(pos, 2000, progress=True) # Burn-in
sampler.reset()
sampler.run_mcmc(state, 10000, progress=True) # Main run

samples = sampler.get_chain(discard=2000, thin=50, flat=True) # Discard burn-in, thin for autocorrelation
M_median, D_median, a_AU_median, e_median, i_deg_median, Omega_deg_median, omega_deg_median, T0_jyear_median = np.median(samples, axis=0)

# Function to calculate state vectors (r, v) from orbital elements at a given time
def calculate_state_vectors_from_elements(orbital_elements_params, t_target_jyear):
    """
    Calculates the 3D position and velocity vectors from orbital elements at a specific time.
    """
    M, D, a_AU, e, i_deg, Omega_deg, omega_deg, T0_jyear = orbital_elements_params

    M_cgs = M * Msun
    a_cm = a_AU * AU
    i_rad = np.radians(i_deg)
    Omega_rad = np.radians(Omega_deg)
    omega_rad = np.radians(omega_deg)

    mu = G * M_cgs # cm^3 s^-2
    n_mean = np.sqrt(mu / a_cm**3) # rad / s

    delta_t_s = (t_target_jyear - T0_jyear) * (365.25 * 24 * 3600)
    M_anom = n_mean * delta_t_s
    M_anom = np.fmod(M_anom, 2 * np.pi)

    E_guess = M_anom
    for _ in range(100):
        f = E_guess - e * np.sin(E_guess) - M_anom
        f_prime = 1 - e * np.cos(E_guess)
        if f_prime == 0: break
        E_new = E_guess - f / f_prime
        if np.abs(E_new - E_guess) < 1e-9:
            E_guess = E_new
            break
        E = E_guess

    # Position in perifocal frame
    x_perifocal = a_cm * (np.cos(E) - e)
    y_perifocal = a_cm * np.sqrt(1 - e**2) * np.sin(E)

    # Velocity in perifocal frame
    Edot = n_mean / (1 - e * np.cos(E))
    vx_perifocal = -a_cm * np.sin(E) * Edot
    vy_perifocal = a_cm * np.sqrt(1 - e**2) * np.cos(E) * Edot

    # Rotation matrices (perifocal to inertial)
    cos_omega, sin_omega = np.cos(omega_rad), np.sin(omega_rad)
    cos_i, sin_i = np.cos(i_rad), np.sin(i_rad)
    cos_Omega, sin_Omega = np.cos(Omega_rad), np.sin(Omega_rad)

    # Full rotation matrix from perifocal to inertial (J2000 equatorial)
    rot_matrix = np.array([
        [cos_Omega * cos_omega - sin_Omega * sin_omega * cos_i, -cos_Omega * sin_omega - sin_Omega * cos_omega * cos_i, sin_Omega * sin_i],
        [sin_Omega * cos_omega + cos_Omega * sin_omega * cos_i, -sin_Omega * sin_omega + cos_Omega * cos_omega * cos_i, -cos_Omega * sin_i],
        [sin_omega * sin_i, cos_omega * sin_i, cos_i]
    ])

    r_perifocal = np.array([x_perifocal, y_perifocal, 0.0]) # 0 for Z in perifocal
    v_perifocal = np.array([vx_perifocal, vy_perifocal, 0.0]) # 0 for Z velocity in perifocal

    r_inertial = rot_matrix @ r_perifocal
    v_inertial = rot_matrix @ v_perifocal

    return r_inertial, v_inertial

# Function to re-derive orbital elements from phase space (for true anomaly)
def phase_space_to_orbital_elements_cgs(r_vec_cm, v_vec_cms, mu_cgs):
    r = np.array(r_vec_cm)
    v = np.array(v_vec_cms)

    r_mag = np.linalg.norm(r)
    v_mag = np.linalg.norm(v)

    h_vec = np.cross(r, v)
    h_mag = np.linalg.norm(h_vec)

    i = np.arccos(h_vec[2] / h_mag)

    K = np.array([0, 0, 1])
    n_vec = np.cross(K, h_vec)
    n_mag = np.linalg.norm(n_vec)

    e_vec = (1 / mu_cgs) * ((v_mag**2 - mu_cgs / r_mag) * r - np.dot(r, v) * v)
    e = np.linalg.norm(e_vec)

    energy = v_mag**2 / 2 - mu_cgs / r_mag
    if energy == 0:
        a = np.inf
    else:
        a = -mu_cgs / (2 * energy)

    Omega = 0.0
    if n_mag != 0:
        Omega = np.arccos(n_vec[0] / n_mag)
        if n_vec[1] < 0:
            Omega = 2 * np.pi - Omega

    omega = 0.0
    if e > 1e-8 and n_mag != 0:
        omega = np.arccos(np.dot(n_vec, e_vec) / (n_mag * e))
        if e_vec[2] < 0:
            omega = 2 * np.pi - omega

    nu = 0.0
    if e > 1e-8:
        cos_nu = np.dot(e_vec, r) / (e * r_mag)
        cos_nu = np.clip(cos_nu, -1, 1)
        nu = np.arccos(cos_nu)
        if np.dot(r, v) < 0:
            nu = 2 * np.pi - nu
    else: # For circular orbits, true anomaly is argument of latitude minus argument of periapsis
        # This part is complex without a fixed definition for circular nu. For this use-case,
        # where we primarily care about its value for a specific time, and e is not zero,
        # the previous part should be sufficient.
        pass

    return a, e, i, Omega, omega, nu

# Get best-fit orbital elements from MCMC
best_fit_params = [M_median, D_median, a_AU_median, e_median, i_deg_median, Omega_deg_median, omega_deg_median, T0_jyear_median]

# Target time: January 1st, 2025
target_time_jyear = Time('2025-01-01').jyear

# Calculate state vectors (position and velocity) at target time
r_vec_2025_cm, v_vec_2025_cms = calculate_state_vectors_from_elements(best_fit_params, target_time_jyear)
mu_cgs_median = G * M_median * Msun # Gravitational parameter with median Mass

# Re-derive orbital elements from state vectors to get true anomaly at 2025-01-01
# Note: a, e, i, Omega, omega should ideally match the median fit parameters from MCMC,
# but nu will be specific to this time.
a_derived_2025_cm, e_derived_2025, i_derived_2025_rad, Omega_derived_2025_rad, omega_derived_2025_rad, nu_derived_2025_rad = \
    phase_space_to_orbital_elements_cgs(r_vec_2025_cm, v_vec_2025_cms, mu_cgs_median)


# Generate orbit for plotting using the median MCMC parameters
# Create a range of times spanning the entire orbit for plotting
orbital_period_sec = 2 * np.pi * np.sqrt(a_AU_median * AU)**3 / (G * M_median * Msun)
orbital_period_yr = orbital_period_sec / (365.25 * 24 * 3600)
# Generate points over one full period, starting from T0
plot_times_jyear = np.linspace(T0_jyear_median, T0_jyear_median + orbital_period_yr, 500)

ra_plot, dec_plot, vz_plot = kepler_model_full(best_fit_params, plot_times_jyear)

# Plot orbit and observations
segments = np.array([[[ra_plot[i], dec_plot[i]], [ra_plot[i + 1], dec_plot[i + 1]]] for i in range(len(ra_plot) - 1)])
fig, ax = plt.subplots(figsize=(8, 6))
lc = LineCollection(segments, cmap=cm.viridis, norm=plt.Normalize(plot_times_jyear.min(), plot_times_jyear.max()))
lc.set_array(np.linspace(0, 1, len(segments))) # Use a normalized array for consistent color mapping over the full orbit
lc.set_linewidth(2)
ax.add_collection(lc)

sc = ax.scatter(delta_ra, delta_dec, c=time, cmap=cm.viridis, norm=plt.Normalize(time.min(), time.max()), s=50, edgecolor='k', label='Observed Data', zorder=4)
cb = fig.colorbar(sc, ax=ax, label='Julian Year (Observation Time)')

# Plot the position on Jan 1st, 2025
ra_2025, dec_2025, _ = kepler_model_full(best_fit_params, np.array([target_time_jyear]))
ax.scatter(ra_2025, dec_2025, marker='*', s=300, color='red', label='Jan 1, 2025', zorder=5)

ax.invert_xaxis()
ax.set_xlabel("ΔR.A. [arcsec]")
ax.set_ylabel("ΔDec. [arcsec]")
ax.set_title("Fitted Keplerian Orbit with Time-Colored Observations")
ax.legend()
ax.grid(True)
ax.axis('equal')
plt.tight_layout()
plt.show()

# Output results
print("\n" + "="*40)
print("Inferred Parameters (from MCMC Fit):")
print(f"Mass of Sgr A*: {M_median:.2e} Msun")
print(f"Distance to Sgr A*: {D_median:.2f} kpc")
print(f"Semi-major axis a = {a_AU_median:.5f} AU")
print(f"Eccentricity e = {e_median:.5f}")
print(f"Inclination i = {i_deg_median:.2f} deg")
print(f"Longitude of Node Ω = {Omega_deg_median:.2f} deg")
print(f"Argument of Periapsis ω = {omega_deg_median:.2f} deg")
print(f"Periapsis Epoch T0 = {T0_jyear_median:.4f} Jyr")
print("="*40)
print(f"Orbital Elements on January 1st, 2025 ({target_time_jyear:.4f} Jyr):")
print(f"Semi-major axis a = {a_derived_2025_cm / AU:.5f} AU") # Should be close to a_AU_median
print(f"Eccentricity e = {e_derived_2025:.5f}") # Should be close to e_median
print(f"Inclination i = {np.degrees(i_derived_2025_rad):.2f} deg") # Should be close to i_deg_median
print(f"Longitude of Node Ω = {np.degrees(Omega_derived_2025_rad):.2f} deg") # Should be close to Omega_deg_median
print(f"Argument of Periapsis ω = {np.degrees(omega_derived_2025_rad):.2f} deg") # Should be close to omega_deg_median
print(f"True Anomaly ν = {np.degrees(nu_derived_2025_rad):.2f} deg") # This value is specific to the time
print("="*40)

Running MCMC with 16 walkers and 8 dimensions...


 97%|█████████▋| 1948/2000 [04:18<00:05,  8.71it/s]